# Claude CLI - Agents et Subagents

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Intermediaire  
**Duree :** 30 min  
**Prerequis :** Notebooks 01-03 completes

## Objectifs d'Apprentissage

- [ ] Comprendre les agents integres (Explore, Plan)
- [ ] Utiliser l'agent Explore pour rechercher du code
- [ ] Utiliser l'agent Plan pour planifier des taches
- [ ] Creer des subagents personnalises
- [ ] Paralleliser des taches avec plusieurs agents

---

## Pourquoi les Agents ?

Les agents permettent de **deleguer des taches complexes** a des instances specialisees de Claude, chacune optimisee pour un type de travail specifique :

- **Efficacite** : Un agent Explore utilise un modele leger (Haiku) pour des recherches rapides
- **Securite** : Les agents en lecture seule ne peuvent pas modifier votre code accidentellement
- **Parallelisation** : Jusqu'a 10 agents peuvent travailler simultanement sur des sous-taches

> **Cas d'usage typique** : Vous demandez "Refactorise ce projet". Claude lance automatiquement des agents Explore pour comprendre la structure, puis un agent Plan pour definir les etapes, avant d'effectuer les modifications.

## 1. Configuration

In [ ]:
import sys
import subprocess
import json
import os

sys.path.insert(0, 'helpers')
from claude_cli import run_claude, verify_installation, print_response

EXAMPLES_DIR = os.path.join(os.getcwd(), 'examples', 'sample_project')
print(f"Claude CLI pret: {verify_installation()}")

## 2. Concept d'Agents

Claude Code dispose d'**agents specialises** qui peuvent effectuer des taches complexes de maniere autonome :

| Agent | Fonction | Outils disponibles |
|-------|----------|-------------------|
| **Explore** | Recherche rapide en lecture seule | Read, Glob, Grep |
| **Plan** | Planification et architecture | Read, Glob, Grep + analyse |
| **General** | Taches generales | Tous les outils |

```
┌─────────────────────────────────────┐
│          Claude Principal           │
│  (Coordonne les taches complexes)   │
└───────────────┬─────────────────────┘
                │
    ┌───────────┼───────────┐
    ▼           ▼           ▼
┌───────┐  ┌───────┐  ┌───────┐
│Explore│  │ Plan  │  │General│
│ Agent │  │ Agent │  │ Agent │
└───────┘  └───────┘  └───────┘
```

> **Note importante** : Les agents Explore et Plan sont **automatiquement** utilises par Claude en mode interactif (`claude` sans `-p`). En mode CLI one-shot (`claude -p "..."`), nous **simulons** leur comportement pour comprendre leur logique.

## 3. Agent Explore

L'agent **Explore** est optimise pour la recherche rapide dans un codebase :

| Caracteristique | Detail |
|-----------------|--------|
| **Mode** | Lecture seule (ne modifie jamais de fichiers) |
| **Modele** | Haiku (rapide, economique) |
| **Outils** | `Glob` (recherche fichiers), `Grep` (recherche contenu), `Read` (lecture) |
| **Timeout** | Plus court que l'agent principal |

### Quand Claude lance-t-il un agent Explore ?

- Quand vous posez une question sur la structure du projet
- Quand vous demandez "trouve tous les fichiers qui..."
- Quand Claude a besoin de contexte avant une modification

### Difference avec une recherche manuelle

```
# Recherche manuelle (vous)
grep -r "def calculate" *.py

# Avec agent Explore (Claude en mode interactif)
"Trouve toutes les fonctions de calcul dans le projet"
→ Claude lance Explore qui utilise Grep intelligemment
→ Explore synthetise les resultats
→ Claude recoit un resume structure
```

En mode interactif, Claude lance automatiquement des agents Explore. En CLI, on peut simuler ce comportement.

In [ ]:
# Simuler une recherche de type Explore
# En pratique, Claude utilise l'agent Explore automatiquement

stdout, stderr, code = run_claude(
    f"""Explore ce projet et reponds a ces questions:
1. Quelles sont les principales fonctions ?
2. Y a-t-il des tests ?
3. Quelles dependances externes sont utilisees ?

Projet:
- main.py : point d'entree
- utils.py : fonctions utilitaires
- tests/test_utils.py : tests unitaires""",
    model="haiku"  # Modele rapide comme un agent Explore
)
print_response(stdout, stderr, code)

In [ ]:
# Recherche de patterns specifiques (comme Grep)
with open(os.path.join(EXAMPLES_DIR, 'utils.py'), 'r') as f:
    utils_content = f.read()

stdout, stderr, code = run_claude(
    f"""Dans ce code, trouve toutes les fonctions qui:
1. Prennent une liste en parametre
2. Retournent un dictionnaire

Code:
{utils_content}""",
    model="haiku"
)
print_response(stdout, stderr, code)

## 4. Agent Plan

L'agent **Plan** aide a structurer des taches complexes avant de les executer :

| Caracteristique | Detail |
|-----------------|--------|
| **Role** | Decomposer une tache en etapes executables |
| **Modele** | Sonnet (raisonnement avance) |
| **Outils** | Read, Glob, Grep + capacite d'analyse |
| **Sortie** | Plan structure avec etapes, risques, dependances |

### Workflow typique avec Plan

```
┌──────────────────────────────────────────────────────────┐
│  Utilisateur: "Ajoute une API REST au projet"            │
└──────────────────────┬───────────────────────────────────┘
                       │
                       ▼
┌──────────────────────────────────────────────────────────┐
│  1. Agent Explore → Analyse structure existante          │
│  2. Agent Plan → Cree plan d'implementation              │
│     - Fichiers a creer                                   │
│     - Ordre des etapes                                   │
│     - Risques identifies                                 │
│  3. Utilisateur → Valide ou ajuste le plan               │
│  4. Agent Principal → Execute le plan valide             │
└──────────────────────────────────────────────────────────┘
```

### Quand utiliser Plan explicitement ?

- **Refactoring majeur** : Avant de toucher a plusieurs fichiers
- **Nouvelle fonctionnalite** : Pour definir l'architecture
- **Migration** : Pour planifier les etapes sans rien casser

In [ ]:
# Demander un plan d'implementation
stdout, stderr, code = run_claude(
    """Je veux ajouter les fonctionnalites suivantes au projet sample_project:

1. Export des resultats en CSV
2. Graphiques avec matplotlib
3. Interface en ligne de commande (CLI)

Cree un plan d'implementation detaille avec:
- Les fichiers a creer/modifier
- L'ordre des etapes
- Les dependances a ajouter
- Les risques potentiels"""
)
print_response(stdout, stderr, code)

In [ ]:
# Plan de refactoring
stdout, stderr, code = run_claude(
    f"""Propose un plan de refactoring pour ce code:

{utils_content}

Objectifs:
- Ameliorer la lisibilite
- Ajouter la gestion d'erreurs
- Permettre l'extension future

Format: etapes numerotees avec justification"""
)
print_response(stdout, stderr, code)

## 5. Subagents Personnalises

Claude Code ne permet pas (encore) de definir des subagents externes via un fichier de configuration. Cependant, vous pouvez **simuler** des agents specialises en utilisant des **system prompts** specifiques.

### Approche par System Prompts

L'idee est de definir des "personas" avec des instructions specifiques :

```python
agents = {
    "security_reviewer": {
        "system_prompt": "Tu es un expert en securite. Analyse uniquement les vulnerabilites.",
        "model": "sonnet"  # Plus precis pour la securite
    },
    "quick_explainer": {
        "system_prompt": "Tu es un tuteur. Explique simplement en 3 phrases max.",
        "model": "haiku"   # Rapide pour les explications courtes
    }
}
```

### Alternatives pour de vrais agents personnalises

| Methode | Description |
|---------|-------------|
| **Skills (.claude/commands/)** | Commandes slash personnalisees |
| **MCP Servers** | Outils externes via Model Context Protocol |
| **Hooks** | Scripts executes avant/apres certaines actions |

> **Note** : Les exemples ci-dessous montrent comment structurer des prompts specialises, pas de vrais subagents Claude.

Vous pouvez definir vos propres agents specialises :

In [ ]:
# Definition d'agents personnalises
custom_agents = {
    "security_reviewer": {
        "description": "Expert en securite",
        "system_prompt": "Tu es un expert en securite applicative. Analyse le code pour identifier les vulnerabilites potentielles (injection, XSS, etc.).",
        "model": "sonnet"
    },
    "performance_analyst": {
        "description": "Expert en performance",
        "system_prompt": "Tu es un expert en optimisation de performance. Identifie les goulots d'etranglement et propose des ameliorations.",
        "model": "sonnet"
    },
    "documentation_writer": {
        "description": "Redacteur de documentation",
        "system_prompt": "Tu es un technical writer. Redige une documentation claire et complete.",
        "model": "haiku"
    }
}

print("Agents definis:")
for name, config in custom_agents.items():
    print(f"  - {name}: {config['description']}")

In [ ]:
# Simuler l'agent security_reviewer
agent_config = custom_agents["security_reviewer"]

stdout, stderr, code = run_claude(
    f"""{agent_config['system_prompt']}

Analyse ce code:
{utils_content}

Identifie les problemes de securite potentiels.""",
    model=agent_config['model']
)
print("=== Agent: security_reviewer ===")
print_response(stdout, stderr, code)

In [ ]:
# Simuler l'agent performance_analyst
agent_config = custom_agents["performance_analyst"]

stdout, stderr, code = run_claude(
    f"""{agent_config['system_prompt']}

Analyse ce code:
{utils_content}

Identifie les problemes de performance et propose des optimisations.""",
    model=agent_config['model']
)
print("=== Agent: performance_analyst ===")
print_response(stdout, stderr, code)

## 6. Parallelisation (Concept)

Claude Code peut lancer **jusqu'a 10 agents en parallele** pour accelerer les taches complexes.

En mode interactif, cela se fait automatiquement. En script, on peut simuler avec des threads :

> **Attention aux couts !** Chaque appel parallele consomme des tokens. Avec 10 agents en parallele, vous multipliez par 10 la consommation. Utilisez `model="haiku"` pour les taches paralleles.

### Bonnes pratiques pour la parallelisation

| A faire | A eviter |
|---------|----------|
| Taches independantes (analyse de fichiers differents) | Taches dependantes (resultat A requis pour B) |
| Modele Haiku pour chaque sous-tache | Modele Opus pour tout |
| Timeout court (30s) par tache | Pas de timeout |
| Agregation des resultats a la fin | Traitement sequentiel deguise |

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def run_agent_task(agent_name, prompt):
    """Execute une tache d'agent et retourne le resultat."""
    start = time.time()
    stdout, stderr, code = run_claude(prompt, model="haiku", timeout=30)
    duration = time.time() - start
    return {
        "agent": agent_name,
        "response": stdout[:500] + "..." if len(stdout) > 500 else stdout,
        "success": code == 0,
        "duration": round(duration, 2)
    }

# Definir les taches
tasks = [
    ("Analyse structure", "Resume la structure de ce code en 3 points"),
    ("Liste fonctions", "Liste toutes les fonctions avec leur signature"),
    ("Identifie types", "Identifie tous les types de donnees utilises")
]

print("Taches definies:")
for name, _ in tasks:
    print(f"  - {name}")

In [ ]:
# Executer en parallele (ATTENTION: consomme des tokens)
# Decommentez pour executer

# with ThreadPoolExecutor(max_workers=3) as executor:
#     futures = {
#         executor.submit(run_agent_task, name, f"{prompt}\n\nCode:\n{utils_content}"): name
#         for name, prompt in tasks
#     }
#     
#     for future in as_completed(futures):
#         result = future.result()
#         print(f"\n=== {result['agent']} ({result['duration']}s) ===")
#         print(result['response'])

print("(Execution parallele desactivee - decommentez pour tester)")

## 7. Exercices

In [ ]:
# EXERCICE 1 : Creez un agent "test_generator"
# qui genere des tests unitaires pour une fonction donnee

test_generator = {
    "system_prompt": "Tu es un expert en tests unitaires Python avec pytest. "
                     "Pour chaque fonction, genere des tests couvrant: "
                     "cas nominal, cas limites (vide, None), et cas d'erreur.",
    "model": "sonnet"
}

# Fonction a tester (de utils.py)
function_to_test = """
def normalize_data(data):
    if not data:
        return []
    min_val = min(data)
    max_val = max(data)
    range_val = max_val - min_val
    if range_val == 0:
        return [0.5] * len(data)
    return [(x - min_val) / range_val for x in data]
"""

# TODO: Completez l'appel a run_claude
# stdout, stderr, code = run_claude(
#     f"{test_generator['system_prompt']}\n\nGenere des tests pour:\n{function_to_test}",
#     model=test_generator['model']
# )
# print_response(stdout, stderr, code)

print("Decommentez le code pour executer l'exercice")

In [ ]:
# EXERCICE 2 : Utilisez l'agent Plan pour planifier
# l'ajout d'une fonctionnalite de votre choix

# Choisissez UNE fonctionnalite parmi:
# - Ajout d'une CLI avec argparse
# - Export des resultats en JSON/CSV
# - Ajout de logging avec le module logging

feature = "Export des resultats en CSV"  # Changez selon votre choix

# TODO: Completez le prompt de planification
# plan_prompt = f"""
# Je veux ajouter cette fonctionnalite au projet sample_project: {feature}
# 
# Contexte du projet:
# - main.py: point d'entree avec fonction main()
# - utils.py: fonctions calculate_statistics(), format_report(), validate_data()
# - tests/test_utils.py: tests existants
#
# Cree un plan d'implementation avec:
# 1. Fichiers a creer/modifier
# 2. Ordre des etapes
# 3. Risques potentiels
# """

# stdout, stderr, code = run_claude(plan_prompt)
# print_response(stdout, stderr, code)

print("Decommentez le code pour executer l'exercice")

## 8. Resume

Dans ce notebook, nous avons appris :

| Concept | Description | Modele |
|---------|-------------|--------|
| **Agent Explore** | Recherche rapide en lecture seule | Haiku |
| **Agent Plan** | Planification et decomposition de taches | Sonnet |
| **Subagents personnalises** | System prompts specialises | Variable |
| **Parallelisation** | Jusqu'a 10 agents simultanement | Haiku recommande |

### Points cles a retenir

1. **Les agents sont automatiques** en mode interactif - vous n'avez pas a les appeler explicitement
2. **Explore = lecture seule** - ne peut jamais modifier de fichiers
3. **Plan avant execution** - pour les taches complexes, demandez d'abord un plan
4. **Parallelisation = cout x N** - utilisez Haiku pour les sous-taches

### Auto-evaluation

Pouvez-vous repondre a ces questions ?

- [ ] Quels outils l'agent Explore peut-il utiliser ?
- [ ] Pourquoi utiliser Haiku plutot que Sonnet pour Explore ?
- [ ] Comment simuler un agent specialise avec un system prompt ?
- [ ] Quelle est la limite de parallelisation de Claude Code ?

### Prochaine etape

Dans le notebook final, nous verrons l'**automatisation avancee** : pipelines, hooks, et integration CI/CD.

-> [05-Claude-CLI-Automatisation.ipynb](05-Claude-CLI-Automatisation.ipynb)